In [1]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [3]:
# !gzip -dc fhvhv_tripdata_2021-01.csv.gz

In [ ]:
!wc -l fhvhv_tripdata_2021-01.csv

In [10]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
df.schema

In [12]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [13]:
import pandas as pd

In [14]:
df_pandas = pd.read_csv('head.csv')

In [ ]:
df_pandas.dtypes

In [ ]:
spark.createDataFrame(df_pandas).schema

Integer - 4 bytes
Long - 8 bytes

In [17]:
from pyspark.sql import types

In [18]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [19]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [20]:
df = df.repartition(24)

In [ ]:
df.write.parquet('fhvhv/2021/01/')

In [22]:
df = spark.read.parquet('fhvhv/2021/01/')

In [ ]:
df.printSchema()

SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [24]:
from pyspark.sql import functions as F

In [ ]:
df.show()

In [26]:
# write a function
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [ ]:
crazy_stuff('B02884')

In [28]:
# create a user defined pyspark function
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [ ]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

In [ ]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

In [ ]:
!head -n 10 head.csv